<a href="https://colab.research.google.com/github/vlad-pirvu/LLM_testing/blob/main/D%26D_Spells3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### I.Dependencies & Imports

In [57]:
import nbformat
from google.colab import _message
import os
import ipynbname

def strip_widgets_metadata():
    # Load current notebook's contents
    # Use a blocking request to get the notebook content
    notebook_content = _message.blocking_request('get_ipynb')['ipynb']
    nb = nbformat.from_dict(notebook_content)

    # More robust removal of 'widgets' metadata globally and from cells
    if 'metadata' in nb and 'widgets' in nb.metadata:
        del nb.metadata['widgets']

    for cell in nb.cells:
        if 'metadata' in cell and 'widgets' in cell.metadata:
            del cell.metadata['widgets']

    # Overwrite the current notebook file
    # Use ipynbname to get the current notebook path
    try:
        notebook_path = ipynbname.path()
        with open(notebook_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        print(f'✅ Cleaned widgets metadata from {notebook_path}')
    except FileNotFoundError:
        print("❌ Could not determine notebook path. Please ensure the notebook is saved.")
    except Exception as e:
        print(f"❌ An error occurred while saving the notebook: {e}")


strip_widgets_metadata()

✅ Cleaned widgets metadata from /fileId=1c0DHWTWa4JdFe2_WNOvDc0G4s4Mymgvd


In [1]:
# Install dependencies
!pip install -q requests beautifulsoup4 sentence-transformers faiss-cpu gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.5 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [39]:
!pip install transformers sentence-transformers

In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr
from tqdm import tqdm
import pandas as pd

### II. Scrape D&D 5e Spells from D&D Wiki

In [3]:
# Step 2: Scrape D&D 5e Spells
BASE_URL = "https://dnd5e.wikidot.com"

In [4]:
def get_spell_links():
    res = requests.get(f"{BASE_URL}/spells")
    soup = BeautifulSoup(res.text, 'html.parser')
    links = [a['href'] for a in soup.select('div#page-content a') if a['href'].startswith('/spell:')]
    return list(set(links))

In [11]:
def scrape_spell(link):
    base_url = "https://dnd5e.wikidot.com"
    if not link.startswith("http"):
        url = base_url + link
    else:
        url = link

    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Try finding the spell name in multiple ways
    title_tag = soup.find('div', {'id': 'page-title'})
    if not title_tag:
        title_tag = soup.find('div', class_='page-title page-header')
    if title_tag:
        # If using class selector, the name is in a <span> inside the div
        if title_tag.name == 'div' and 'page-header' in title_tag.get('class', []):
            name = title_tag.find('span').text.strip()
        else:
            name = title_tag.text.strip()
    else:
        print(f"⚠️ Skipping {url}: no title found.")
        return None

    # Skip Unearthed Arcana spells
    if name.endswith(" UA") or name.endswith(" (UA)"):
        return None

    # Get the spell content
    content_tag = soup.find('div', {'id': 'page-content'})
    if not content_tag:
        print(f"⚠️ Skipping {url}: no page-content found.")
        return None
    content = content_tag.text.strip()

    return {'Url': url, 'Name': name, 'content': content}


In [13]:
spell_links = get_spell_links()
print(f"Found {len(spell_links)} spells. Scraping...")

# Collect spells
spells_data = []
for link in tqdm(spell_links, desc="Scraping spells"):
#    full_url = BASE_URL + link
    spell = scrape_spell(link)
    if spell:
        spells_data.append(spell)

Found 580 spells. Scraping...


Scraping spells:  43%|████▎     | 251/580 [01:06<06:14,  1.14s/it]

⚠️ Skipping https://dnd5e.wikidot.com/spell:blade-barrier: no title found.


Scraping spells: 100%|██████████| 580/580 [02:30<00:00,  3.85it/s]


### III. Format Spells DataFrame

In [29]:
# Convert to DataFrame
spells_df = pd.DataFrame(spells_data)
print(f"✅ Collected {len(spells_df)} spells.")
spells_df.head()

✅ Collected 529 spells


,Url,Name,content
0,https://dnd5e.wikidot.com/spell:beacon-of-hope,Beacon of Hope,Source: Player's Handbook\n3rd-level abjuratio...
1,https://dnd5e.wikidot.com/spell:nathairs-mischief,Nathair's Mischief,Source: Fizban's Treasury of Dragons\n2nd-leve...
2,https://dnd5e.wikidot.com/spell:shocking-grasp,Shocking Grasp,Source: Player's Handbook\nEvocation cantrip\n...
3,https://dnd5e.wikidot.com/spell:dream-of-the-b...,Dream of the Blue Veil,Source: Tasha's Cauldron of Everything\n7th-le...
4,https://dnd5e.wikidot.com/spell:chain-lightning,Chain Lightning,Source: Player's Handbook\n6th-level evocation...


In [15]:
spells_df.shape

(529, 3)

In [16]:
def split_spell_content(row):
    #print("------")
    #print(row.name)
    #print(row['name'])

    lines = row['content'].split('\n')
    lines = [line.strip() for line in lines if line.strip()]  # Clean up empty lines and whitespace

    # Safe defaults
    source = lines[0].replace("Source", "").replace("source","").lstrip(',.:;-') if len(lines) > 0 else ''
    spell_level_type = lines[1].replace("Level", "").replace("level", "").replace("-","").replace("st","").replace("nd","").replace("rd","").replace("th","").lstrip(',.:;-') if len(lines) > 1 else ''
    casting_time = lines[2].replace("Casting Time", "").replace("Casting time","").replace("casting time","").lstrip(',.:;-') if len(lines) > 2 else ''
    spell_range = lines[3].replace("Range", "").replace("range","").lstrip(',.:;-') if len(lines) > 3 else ''
    components = lines[4].replace("Components", "").replace("components","").lstrip(',.:;-') if len(lines) > 4 else ''
    duration = lines[5].replace("Duration", "").replace("duration","").lstrip(',.:;-') if len(lines) > 5 else ''

    # Split spell_level_type into level and type
    if 'cantrip' in spell_level_type.lower():
        level, school = '0', spell_level_type.replace('cantrip', '').strip()
    elif ' ' in spell_level_type:
        level, school = spell_level_type.split(' ', 1)
    else:
        level, school = '', spell_level_type


    #Extract description
    mashed = row['content'].split('Duration',1)[1]
    mashed = mashed.split('\n',1)[1].strip().lower()

    if 'at higher levels' in mashed:
        description = mashed.split('at higher levels')[0].strip()
        upcastingMashed = mashed.split('at higher levels')[1].lstrip(',.:;-').strip()
        upcasting = upcastingMashed.split('spell lists')[0].strip()
        spellLists = upcastingMashed.split('spell lists')[1].lstrip(',.:;-').strip()
    else:
        description = mashed.split('spell lists')[0].strip()
        mashed = mashed.replace(":",".")
        spellLists = mashed.split('spell lists')[1].lstrip(',.:;-').strip()
        upcasting = ""

    return pd.Series({
        'Source': source,
        'Level': level,
        'School': school,
        'Casting Time': casting_time,
        'Range': spell_range,
        'Components': components,
        'Duration': duration,
        'Description': description,
        'Upcasting': upcasting,
        'Spell Lists': spellLists
    })


In [17]:
# Apply the split function to the spells DataFrame
spell_df_split = spells_df.join(spells_df.apply(split_spell_content, axis=1))

In [ ]:
spell_df_split.head()

,url,name,content,Source,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists
0,https://dnd5e.wikidot.com/spell:illusory-dragon,Illusory Dragon,Source: Xanathar's Guide to Everything\n8th-le...,Xanathar's Guide to Everything,8,illusion,1 action,120 feet,S,"Concentration, up to 1 minute",by gathering threads of shadow material from t...,,wizard
1,https://dnd5e.wikidot.com/spell:chaos-bolt,Chaos Bolt,Source: Xanathar's Guide to Everything\n1st-le...,Xanathar's Guide to Everything,1,evocation,1 action,120 feet,"V, S",Instantaneous,"you hurl an undulating, warbling mass of chaot...",when you cast this spell using a spell slot of...,sorcerer
2,https://dnd5e.wikidot.com/spell:harm,Harm,Source: Player's Handbook\n6th-level necromanc...,Player's Handbook,6,necromancy,1 action,60 feet,"V, S",Instantanous,you unleash a virulent disease on a creature t...,,cleric
3,https://dnd5e.wikidot.com/spell:power-word-heal,Power Word: Heal,Source: Player's Handbook\n9th-level evocation...,Player's Handbook,9,evocation,1 action,Touch,"V, S",Instantaneous,a wave of healing energy washes over a creatur...,,"bard, cleric (optional)"
4,https://dnd5e.wikidot.com/spell:time-stop,Time Stop,Source: Player's Handbook\n9th-level transmuta...,Player's Handbook,9,transmutation,1 action,Self,V,Instantaneous,you briefly stop the flow of time for everyone...,,"sorcerer, wizard"


In [ ]:
spell_df_split.describe()

,url,name,content,Source,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists
count,530,530,530,530,530,530,530,530,530,530,530,530,530
unique,530,530,530,17,14,34,20,37,276,32,530,156,161
top,https://dnd5e.wikidot.com/spell:polymorph,Polymorph,Source: Player's Handbook\n4th-level transmuta...,Player's Handbook,2,evocation,1 action,60 feet,"V, S",Instantaneous,this spell transforms a creature that you can ...,,wizard
freq,1,1,1,361,86,90,395,96,161,148,1,329,41


In [18]:
non_numeric_levels = spell_df_split[~spell_df_split['Level'].str.isdigit()]['Level'].unique()
print("Non-numeric values in the 'Level' column:", non_numeric_levels)

Non-numeric values in the 'Level' column: []


In [19]:
spell_df_split = spell_df_split.drop('content', axis=1)

In [24]:
spell_df_split = spell_df_split.rename(columns={'name': 'Name', 'Url':"URL"})
print(spell_df_split.columns)

Index(['URL', 'Name', 'Source', 'Level', 'School', 'Casting Time', 'Range',
       'Components', 'Duration', 'Description', 'Upcasting', 'Spell Lists'],
      dtype='object')


In [25]:
desired_column_order = ['Name', 'Level', 'School', 'Casting Time', 'Range', 'Components', 'Duration', 'Description', 'Upcasting', 'Spell Lists', 'Source', 'URL']

In [27]:
finalSpells = spell_df_split[desired_column_order]

In [ ]:
finalSpells.head()

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL
0,Illusory Dragon,8,illusion,1 action,120 feet,S,"Concentration, up to 1 minute",by gathering threads of shadow material from t...,,wizard,Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:illusory-dragon
1,Chaos Bolt,1,evocation,1 action,120 feet,"V, S",Instantaneous,"you hurl an undulating, warbling mass of chaot...",when you cast this spell using a spell slot of...,sorcerer,Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:chaos-bolt
2,Harm,6,necromancy,1 action,60 feet,"V, S",Instantanous,you unleash a virulent disease on a creature t...,,cleric,Player's Handbook,https://dnd5e.wikidot.com/spell:harm
3,Power Word: Heal,9,evocation,1 action,Touch,"V, S",Instantaneous,a wave of healing energy washes over a creatur...,,"bard, cleric (optional)",Player's Handbook,https://dnd5e.wikidot.com/spell:power-word-heal
4,Time Stop,9,transmutation,1 action,Self,V,Instantaneous,you briefly stop the flow of time for everyone...,,"sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:time-stop


In [ ]:
finalSpells = finalSpells.sort_values(by='Level')

In [28]:
finalSpells.head(50)

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL
0,Beacon of Hope,3,abjuration,1 action,30 feet,"V, S","Concentration, up to 1 minute",this spell bestows hope and vitality. choose a...,,cleric,Player's Handbook,https://dnd5e.wikidot.com/spell:beacon-of-hope
1,Nathair's Mischief,2,Illusion,1 Action,60ft,"S, M (a piece of crust from an apple pie)","Concentration, up to 1 minute",you fill a 20-foot cube you can see within ran...,,"bard, sorcerer, wizard",Fizban's Treasury of Dragons,https://dnd5e.wikidot.com/spell:nathairs-mischief
2,Shocking Grasp,0,Evocation,1 action,Touch,"V, S",Instantaneous,lightning springs from your hand to deliver a ...,the spell’s damage increases by 1d8 when you r...,"artificer, sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:shocking-grasp
3,Dream of the Blue Veil,7,conjuration,10 minutes,20 feet,"V, S, M (a magic item or a willing creature f...",6 hours,you and up to eight willing creatures within r...,,"bard (optional), sorcerer (optional), warlock ...",Tasha's Cauldron of Everything,https://dnd5e.wikidot.com/spell:dream-of-the-b...
4,Chain Lightning,6,evocation,1 action,150 feet,"V, S, M (a bit of fur; a piece of amber, glas...",Instantaneous,you create a bolt of lightning that arcs towar...,when you cast this spell using a spell slot of...,"sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:chain-lightning
5,Forcecage,7,evocation,1 action,100 feet,"V, S, M (ruby dust worth 1,500 gp)",1 hour,"an immobile, invisible, cube-shaped prison com...",,"bard, warlock, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:forcecage
6,Mass Suggestion,6,enchantment,1 action,60 feet,"V, M (a snake’s tongue and either a bit of ho...",24 hours,you suggest a course of activity (limited to a...,when you cast this spell using a 7th-level spe...,"bard, sorcerer, warlock, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:mass-suggestion
7,Evard's Black Tentacles,4,conjuration,1 action,90 feet,"V, S, M (a piece of tentacle from a giant oct...","Concentration, up to 1 minute","squirming, ebony tentacles fill a 20-foot squa...",,wizard,Player's Handbook,https://dnd5e.wikidot.com/spell:evards-black-t...
8,Flaming Sphere,2,conjuration,1 action,60 feet,"V, S, M (a bit of tallow, a pinch of brimston...","Concentration, up to 1 minute",a 5-foot-diameter sphere of fire appears in an...,when you cast this spell using a spell slot of...,"druid, sorcerer (optional), wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:flaming-sphere
9,Greater Restoration,5,abjuration,1 action,Touch,"V, S, M (diamond dust worth at least 100 gp, ...",Instantaneous,you imbue a creature you touch with positive e...,,"artificer, bard, cleric, druid, ranger (optional)",Player's Handbook,https://dnd5e.wikidot.com/spell:greater-restor...


### IV. The real deal

In [30]:
#Step IV.1 - Prepare the data to feed to the model

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Create a 'searchable_text' column
finalSpells['searchable_text'] = finalSpells.apply(lambda row: f"{row['Name']} {row['Level']} {row['School']} {row['Casting Time']} {row['Range']} {row['Components']} {row['Duration']} {row['Description']} {row['Spell Lists']} {row['Source']}", axis=1)

# Show example
print(finalSpells[['Name', 'searchable_text']].head())


                     Name                                    searchable_text
0          Beacon of Hope  Beacon of Hope 3 abjuration  1 action  30 feet...
1      Nathair's Mischief  Nathair's Mischief 2 Illusion  1 Action  60ft ...
2          Shocking Grasp  Shocking Grasp 0 Evocation  1 action  Touch  V...
3  Dream of the Blue Veil  Dream of the Blue Veil 7 conjuration  10 minut...
4         Chain Lightning  Chain Lightning 6 evocation  1 action  150 fee...


<ipython-input-30-d64f5373800e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalSpells['searchable_text'] = finalSpells.apply(lambda row: f"{row['Name']} {row['Level']} {row['School']} {row['Casting Time']} {row['Range']} {row['Components']} {row['Duration']} {row['Description']} {row['Spell Lists']} {row['Source']}", axis=1)


In [31]:
#Step IV.2 - Embed the searchable text

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all spells
embeddings = model.encode(finalSpells['searchable_text'].tolist(), show_progress_bar=True)

# Convert to numpy array for FAISS
embeddings_np = np.array(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [32]:
#Step IV.3 - Create FAISS index

index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

In [33]:
#Step IV.4 v2 - Create a function to query the model

def hybrid_search(query, level=None, spell_class=None, k=5):
    filtered_df = finalSpells.copy()

    # Optional filter for level
    if level is not None:
        print(f"Filtering for level {level}")
        filtered_df = filtered_df[filtered_df['Level'] == str(level)]
        print(f"Remaining after level filter: {len(filtered_df)}")

    # Optional filter for class (case-insensitive)
    if spell_class is not None:
        print(f"Filtering for class {spell_class}")
        mask = filtered_df['Spell Lists'].str.lower().str.contains(spell_class.lower(), na=False)
        filtered_df = filtered_df[mask]
        print(f"Remaining after class filter: {len(filtered_df)}")

    if filtered_df.empty:
        return pd.DataFrame()  # Return empty DataFrame for consistency

    # Step 2: Embed the filtered subset
    filtered_texts = filtered_df['searchable_text'].tolist()
    filtered_embeddings = model.encode(filtered_texts, convert_to_numpy=True)

    d = filtered_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(filtered_embeddings)

    query_embedding = model.encode([query], convert_to_numpy=True)

    D, I = index.search(query_embedding, min(k, len(filtered_df)))

    results = filtered_df.iloc[I[0]].reset_index(drop=True)
    return results


In [36]:
results = hybrid_search("burn enemies", level=3, spell_class="sorcerer", k=5)

if not results.empty:
    print(results[['Name', 'Level','Description']])
else:
    print("No spells found for the given level and class.")


Filtering for level 3
Remaining after level filter: 75
Filtering for class sorcerer
Remaining after class filter: 34
                    Name Level  \
0               Fireball     3   
1           Flame Arrows     3   
2             Antagonize     3   
3    Ashardalon's Stride     3   
4  Melf's Minute Meteors     3   

                                         Description  
0  a bright streak flashes from your pointing fin...  
1  you touch a quiver containing arrows or bolts....  
2  you whisper magical words that antagonize one ...  
3  the billowing flames of a dragon blast from yo...  
4  you create six tiny meteors in your space. the...  


In [ ]:
results.head()

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL,searchable_text
0,Suggestion,2,enchantment,1 action,30 feet,"V, M (a snake’s tongue and either a bit of ho...","Concentration, up to 8 hours",you suggest a course of activity (limited to a...,,"bard, sorcerer, warlock, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:suggestion,Suggestion 2 enchantment 1 action 30 feet V...
1,Hold Person,2,enchantment,1 action,60 feet,"V, S, M (a small, straight piece of iron)","Concentration, up to 1 minute",choose a humanoid that you can see within rang...,when you cast this spell using a spell slot of...,"bard, cleric, druid, sorcerer, warlock, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:hold-person,Hold Person 2 enchantment 1 action 60 feet ...
2,Maximillian's Earthen Grasp,2,transmutation,1 action,30 feet,"V, S, M (a miniature hand sculpted from clay)","Concentration, up to 1 minute",you choose a 5-foot-square unoccupied space on...,,"sorcerer, wizard",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:maximillians-e...,Maximillian's Earthen Grasp 2 transmutation 1...
3,Enlarge/Reduce,2,transmutation,1 action,30 feet,"V, S, M (a pinch of powdered iron)","Concentration, up to 1 minute",you cause a creature or an object you can see ...,,"artificer, bard (optional), druid (optional), ...",Player's Handbook,https://dnd5e.wikidot.com/spell:enlarge-reduce,Enlarge/Reduce 2 transmutation 1 action 30 f...
4,Dust Devil,2,conjuration,1 action,60 feet,"V, S, M (a pinch of dust)","Concentration, up to 1 minute",choose an unoccupied 5-foot cube of air that y...,when you cast this spell using a spell slot of...,"druid, sorcerer, wizard",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:dust-devil,Dust Devil 2 conjuration 1 action 60 feet V...


In [40]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load model and tokenizer once
llm_model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
model_llm = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)

def llm_rewrite_query_hf(user_query):
    prompt = f"Rewrite this D&D spell search query to focus it for filtering a spell database:\n\n{user_query}\n\nRewritten query:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)
    outputs = model_llm.generate(**inputs, max_length=50)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return rewritten


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [41]:
def hybrid_search_with_llm_hf(query, level=None, spell_class=None, k=5):
    rewritten_query = llm_rewrite_query_hf(query)
    print(f"Original query: {query}")
    print(f"Rewritten query: {rewritten_query}")

    level = str(level) if level is not None else None
    spell_class = spell_class if spell_class != "" else None

    filtered_df = finalSpells.copy()
    if level:
        filtered_df = filtered_df[filtered_df['Level'] == level]
    if spell_class:
        filtered_df = filtered_df[filtered_df['Spell Lists'].str.contains(spell_class, case=False, na=False)]

    if filtered_df.empty:
        return f"No spells found for level={level} and class={spell_class}"

    filtered_texts = filtered_df['searchable_text'].tolist()
    filtered_embeddings = model.encode(filtered_texts, convert_to_numpy=True)

    d = filtered_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(filtered_embeddings)

    query_embedding = model.encode([rewritten_query], convert_to_numpy=True)
    D, I = index.search(query_embedding, min(k, len(filtered_df)))

    results = filtered_df.iloc[I[0]].reset_index(drop=True)
    return results


In [42]:
def summarize_spells_with_llm(spells_df, user_query, max_spells=5):
    # Format the spells info into a plain text summary prompt
    spells_summary = ""
    for i, row in spells_df.head(max_spells).iterrows():
        spells_summary += f"- {row['Name']} (Level {row['Level']} {row['School']}): {row['Description'][:150].strip()}...\n"

    prompt = f"User query: {user_query}\n\nBased on the following D&D spells, provide a friendly summary highlighting the best options:\n\n{spells_summary}\nSummary:"

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model_llm.generate(**inputs, max_length=150, do_sample=True, top_p=0.95, temperature=0.7)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


In [43]:
def hybrid_search_and_summarize(query, level=None, spell_class=None, k=5):
    results_df = hybrid_search_with_llm_hf(query, level, spell_class, k)

    if isinstance(results_df, str):
        # means "No spells found" message or error string
        return results_df

    summary = summarize_spells_with_llm(results_df, query, max_spells=k)
    return summary


In [44]:
summary = hybrid_search_and_summarize("burn enemies", level=2, spell_class="sorcerer", k=5)
print(summary)


Original query: burn enemies
Rewritten query: Burn enemies is a list of all the spells that are in the database.
Scorching Ray (Level 2 evocation): you create three rays of fire and hurl them at targets within range. you can hurl them at one target or several. make a ranged spell attack for each r...


In [45]:
import gradio as gr

def gradio_spell_search(query, level, spell_class, k):
    level = str(level) if level is not None else None
    spell_class = spell_class if spell_class != "" else None

    results = hybrid_search(query, level=level, spell_class=spell_class, k=int(k))

    if isinstance(results, str):  # No results found
        return results
    else:
        # Format results nicely
        return results[['Name', 'Level', 'School', 'Spell Lists', 'Description', 'Source', 'URL']].to_markdown(index=False)

# Gradio interface
gr.Interface(
    fn=gradio_spell_search,
    inputs=[
        gr.Textbox(label="Search Query", placeholder="Describe what you're looking for..."),
        gr.Number(label="Spell Level (optional)", value=None),
        gr.Textbox(label="Class Filter (optional)", placeholder="e.g. wizard, sorcerer..."),
        gr.Number(label="Top K Results", value=5)
    ],
    outputs=gr.Markdown(label="Matching Spells"),
    title="D&D 5e Spell Search",
    description="Search for spells based on a natural language query, with optional level and class filtering."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://061df415f9a60e77a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [54]:
!pip install ipynbname

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.7 MB/s eta 0:00:00
